# Bruins/Celtics Game Analysis

This file examines post-game entries at North Station station in comparison to expectations.

## Setup

In [2]:
# Libraries.
import datetime as dt
import matplotlib, matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as snsb

# Feature libraries.
from features import date

In [3]:
# Package settings.

# Matplotlib.
%matplotlib inline

# Seaborn.
sns.set_style('darkgrid')

In [4]:
# Other settings.

# Stations near TD Garden for analysis. Just use names (not locationid) as a surrogate key for station.
td_stations = ['North Station']

# Base units of timedeltas.
timedelta_base = 3.6e12

# Default figure size.
default_figsize = (12,8)

# Day of week mapping.
days_of_week = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']

## Data

### Read Bruins

In [107]:
# Read Red Sox home games.
bruins = pd.DataFrame.from_csv("../../../data/bruins/home.csv").reset_index(drop=False)

# Drop home/playoff columns.
bruins.drop(['Home','Playoff'], axis=1, inplace=True)

# Rename other columns to make joining easy.
bruins.rename(columns={'Datetime':'game_datetime', 'Opponent':'opponent'}, inplace=True)

# Get standalone game date.
bruins['game_date'] = pd.to_datetime(pd.DatetimeIndex(bruins.game_datetime).date)

# Turn game date/time into an appropriate object.
# bruins.game_datetime = pd.to_datetime(bruins.game_datetime)

# Add game time column.
bruins['game_time'] = pd.DatetimeIndex(bruins.game_datetime).time

# Add sport column for later merging.
bruins['team'] = 'Bruins'

In [ ]:
bruins.head()

### Read Celtics

In [105]:
# Read Red Sox home games.
celtics = pd.DataFrame.from_csv("../../../data/celtics/home.csv").reset_index(drop=False)

# Drop home/playoff columns.
celtics.drop(['Home','Playoff'], axis=1, inplace=True)

# Rename other columns to make joining easy.
celtics.rename(columns={'Datetime':'game_datetime', 'Opponent':'opponent'}, inplace=True)

# Get standalone game date. There must be a better way, but this works for now.
celtics['game_date'] = pd.to_datetime(pd.DatetimeIndex(celtics.game_datetime).date)

# Turn game date/time into an appropriate object.
# celtics.game_datetime = pd.to_datetime(celtics.game_datetime)

# Add game time column.
celtics['game_time'] = pd.DatetimeIndex(celtics.game_datetime).time

# Add sport column for later merging.
celtics['team'] = 'Celtics'

celtics.head()

### Read Entries

In [6]:
# Import gatecount data.
def get_data():
    return pd.DataFrame.from_csv("../../../data/mbta.csv").reset_index()
gatecount = date.init(get_data())

# Restrict to desired stations and the columns that we need.
td_gatecount = gatecount.ix[gatecount.name.isin(td_stations),['locationid','name','service_day','service_datetime','entries']].reset_index(drop=True)

# Turn days/dates into datetimes.
td_gatecount.service_day = pd.to_datetime(td_gatecount.service_day)
td_gatecount.service_datetime = pd.to_datetime(td_gatecount.service_datetime)

# Get time of day.
td_gatecount['service_time'] = pd.DatetimeIndex(td_gatecount.service_datetime).time

# Add day of week.
td_gatecount = date.add_day_of_week(td_gatecount.copy())

### Merge

In [116]:
# Merge all games.
games = bruins.append(celtics)

# Get teams set for later traversal.
teams = set(games.team)

In [112]:
# Perform merge.
td_games = td_gatecount.merge(games, left_on='service_day', right_on='game_date', how='left')

# Flag days with games.
td_games['game_played'] = td_games.team.isnull() == False

# Calculate delta from game start to entries record time.
td_games['time_since_game_start'] = td_games.service_datetime - td_games.game_datetime

In [ ]:
td_games[td_games.game_played].head()

## Game Exploration

Let's figure out when games happen.

In [117]:
# Copy DF for this analysis.
games_only = games.copy()
# Add weekday.
games_only['day_of_week'] = pd.DatetimeIndex(games_only.game_date).weekday

When do games occur during the week? We might expect a consistent pattern as the teams share an arena and their seasons largely overlap. **Monday is 0.**

In [118]:
# Summarize by team-DoW.
games_by_team_dow = games_only.groupby(['team','day_of_week']).agg({'game_time':len})
games_by_team_dow.columns = ['games']
games_by_team_dow

games
team    day_of_week       
Bruins  0               17
        1               20
        2                8
        3               29
        4                8
        5               38
        6                8
Celtics 0                9
        1                4
        2               43
        3                2
        4               41
        5                6
        6               17

Aha! Hockey mostly takes Tuesday/Thursday/Saturday; Basketball takes Wednesday/Friday/Sunday. Looks like the Bruins have a slight edge on Monday too.

What about during the day?

In [121]:
# Group and aggregate by time.
games_by_time = games_only.groupby(['team','game_time']).agg({'opponent':len})
games_by_time.columns = ['games']
games_by_time

games
team    game_time       
Bruins  12:30:00       5
        13:00:00      19
        15:00:00       5
        17:30:00       2
        19:00:00      79
        19:30:00      13
        20:00:00       5
Celtics 12:30:00       1
        13:00:00      12
        15:00:00       1
        18:00:00       8
        18:30:00       1
        19:00:00       3
        19:30:00      85
        20:00:00      11

Now combine the above.

In [128]:
# Group and aggregate by rounded time.
games_by_datetime = games_only.groupby(['team','day_of_week','game_time']).agg({'opponent':len})
games_by_datetime.columns = ['games']
games_by_datetime

games
team    day_of_week game_time       
Bruins  0           13:00:00       4
                    15:00:00       1
                    19:00:00       8
                    19:30:00       2
                    20:00:00       2
        1           19:00:00      18
                    19:30:00       2
        2           19:00:00       3
                    19:30:00       3
                    20:00:00       2
        3           19:00:00      26
                    19:30:00       3
        4           13:00:00       1
                    19:00:00       5
                    19:30:00       1
                    20:00:00       1
        5           12:30:00       3
                    13:00:00      14
                    15:00:00       2
                    17:30:00       2
                    19:00:00      17
        6           12:30:00       2
                    15:00:00       2
                    19:00:00       2
                    19:30:00       2
Celtics 0           19:30:00       8
                    20:00:00       1
        1           13:00:00       1
                    19:00:00       1
                    19:30:00       2
        2           13:00:00       1
                    19:30:00      37
                    20:00:00       5
        3           20:00:00       2
        4           13:00:00       2
                    19:00:00       1
                    19:30:00      36
                    20:00:00       2
        5           12:30:00       1
                    13:00:00       2
                    19:30:00       2
                    20:00:00       1
        6           13:00:00       6
                    15:00:00       1
                    18:00:00       8
                    18:30:00       1
                    19:00:00       1

Overall, things look *pretty* consistent. For modeling, we'll have to find a way to handle some of the very uncommon cases (e.g., a Celtics game on a Tuesday).

## Game/Entries Exploration

TODO